# Evaluation Notebook

This notebook provides a universal evaluation system for comparing any AI agent architectures by analyzing conversation traces from Langfuse. This completely reusable framework enables teams to objectively compare any agentic systems across different domains, tasks, and architectures using standardized metrics and Langfuse trace data.


## What It Does
Evaluates Any Agent Framework:

- Single agents, multi-agent systems, agent swarms, or custom architectures
- Works across any domain or use case
- Just provide Langfuse trace IDs for any agent configuration


## How It Works
All you need is:

- Trace IDs: From your agentic frameworks instrumented with Langfuse
- Langfuse API Keys: Your public and secret keys for data access
- Ground Truth Data: Expected behaviors/outputs for comparison

Calculated Metrics:

- missed_tool_pct: Percentage of required tools not called
- incorrect_tool_pct: Percentage of incorrectly used tools
- tools_args_acc: Accuracy of arguments passed to tools
- answer_relevancy: Relevance of agent responses to user queries
- compliance_score: Adherence to domain-specific policies (0.0-1.0)
- compliance_reasoning: Detailed explanation of compliance assessment
- latency: Total response time
- input_tokens: Number of input tokens consumed
- output_tokens: Number of output tokens generated



In [1]:
import requests
import base64
import json
from datetime import datetime
from evaluation import *
import pandas as pd
import re
pd.set_option('display.max_colwidth', None)
import warnings
warnings.filterwarnings('ignore')

In [2]:
compliance_rules = """
## Flight Change Policy:
Basic economy tickets are generally non-changeable unless within a 24-hour grace period of booking, in which case they can be canceled for a full refund and a new flight can be booked.
Changes to higher-tier tickets (e.g., premium economy, business class) may incur a change fee, which varies based on the fare class and the time remaining until departure.
Destination changes are not permitted for existing bookings; a cancellation and new booking are required.
Changes are subject to availability on the requested new flight.
## Cancellation Policy:
Full refunds for cancellations are only available within the 24-hour grace period from the time of booking.
Cancellations outside the grace period may result in a partial refund or flight credit, depending on the ticket type and airline terms.
## Baggage Policy:
Checked baggage fees apply to most basic economy tickets.
Carry-on baggage must adhere to specific size and weight restrictions.
"""

## Functions

In [3]:
public_key="ADD-Langfuse-Public-Key"
secret_key="ADD-Langfuse-Secret-Key"
host="https://us.cloud.langfuse.com"

with open('../data/tau-bench/tau_bench/envs/airline/tasks_singleturn.json', 'r') as file:
    gt_data = json.load(file)

In [6]:
def get_trace_with_observations(trace_id, public_key, secret_key, host):
    """
    Get trace and observations using direct API calls
    """
    # Create basic auth header
    credentials = f"{public_key}:{secret_key}"
    encoded_credentials = base64.b64encode(credentials.encode()).decode()
    
    headers = {
        "Authorization": f"Basic {encoded_credentials}",
        "Content-Type": "application/json"
    }
    
    # Get trace data
    trace_url = f"{host}/api/public/traces/{trace_id}"
    
    try:
        print("Fetching trace data...")
        trace_response = requests.get(trace_url, headers=headers)
        
        if trace_response.status_code == 200:
            trace_data = trace_response.json()
            
        else:
            print(f"❌ Error fetching trace: {trace_response.status_code}")
            print(f"Response: {trace_response.text}")
            return None
            
        # Get observations
        observations_url = f"{host}/api/public/observations"
        params = {"traceId": trace_id}
        
        obs_response = requests.get(observations_url, headers=headers, params=params)
        
        if obs_response.status_code == 200:
            observations_data = obs_response.json()
            observations = observations_data.get('data', [])
            
            print(f"✅ Found {len(observations)} observations")
            
            for i, obs in enumerate(observations, 1):
                
                # Usage and cost info
                if obs.get('usage'):
                    usage = obs['usage']
                    
                # if obs.get('calculatedTotalCost'):
                #     print(f"Total Cost: \${obs['calculatedTotalCost']}")
                    
                # Input (truncated)
                if obs.get('input'):
                    input_str = json.dumps(obs['input'])
                    
                # Output (truncated)
                if obs.get('output'):
                    output_str = json.dumps(obs['output'])
                    
        else:
            print(f"❌ Error fetching observations: {obs_response.status_code}")
            print(f"Response: {obs_response.text}")
            
        return trace_data, observations if 'observations' in locals() else []
        
    except Exception as e:
        print(f"❌ Exception occurred: {e}")
        return None, []



In [7]:
def extract_tools_info_from_langfuse(trace_data, observations):
    """
    Extract tool usage information from Langfuse trace data and observations
    
    Args:
        trace_data: The trace data from Langfuse
        observations: The list of observations from Langfuse
        
    Returns:
        dict: A dictionary containing tool usage information and overall metrics
    """
    # Initialize the result dictionary
    result = {
        'user_messages': [],
        'assistant_messages': [],
        'tools_list': [],
        'tools_args': [],
        'tools_responses': [],
        'total_latency': trace_data.get('latency', 0),
        'input_tokens': 0,
        'output_tokens': 0
    }
    
    # First, sort observations by startTime to maintain chronological order
    sorted_observations = sorted(observations, key=lambda x: x.get('startTime', ''))
    
    # Process observations to extract tools and messages
    for obs in sorted_observations:
        # Add token usage
        if 'usageDetails' in obs:
            result['input_tokens'] += obs['usageDetails'].get('input', 0)
            result['output_tokens'] += obs['usageDetails'].get('output', 0)
        
        # Extract tools from TOOL observations
        if obs.get('type') == 'TOOL':
            tool_name = obs.get('name', '').replace('execute_tool ', '')
            if tool_name:
                result['tools_list'].append(tool_name)
                
                # Get tool arguments
                args = {}
                if obs.get('input') and len(obs['input']) > 0 and 'content' in obs['input'][0]:
                    content = obs['input'][0]['content']
                    try:
                        args = json.loads(content)
                    except (json.JSONDecodeError, TypeError):
                        args = content
                result['tools_args'].append(args)
                
                # Get tool response
                response = ""
                if obs.get('output') and 'message' in obs['output']:
                    message = obs['output']['message']
                    try:
                        # If it looks like JSON, try to extract text content
                        if isinstance(message, str) and message.startswith('[{'):
                            parsed = json.loads(message)
                            if isinstance(parsed, list):
                                for item in parsed:
                                    if isinstance(item, dict) and 'text' in item:
                                        response += item['text']
                        else:
                            response = message
                    except (json.JSONDecodeError, TypeError):
                        response = str(message)
                result['tools_responses'].append(response)
        
        # Extract user messages and intermediate assistant messages from conversation history
        if obs.get('input'):
            for message in obs['input']:
                content = message.get('content', '')
                
                if message.get('role') == 'user':
                    user_text = extract_text_content(content)
                    if user_text and user_text not in result['user_messages']:
                        result['user_messages'].append(user_text)
                
                elif message.get('role') == 'assistant':
                    assistant_text = extract_text_content(content)
                    if assistant_text and assistant_text not in result['assistant_messages']:
                        result['assistant_messages'].append(assistant_text)
        
        # Extract final assistant responses from output field
        if (obs.get('type') == 'GENERATION' or obs.get('type') == 'AGENT') and 'output' in obs:
            if 'message' in obs['output']:
                assistant_text = extract_text_content(obs['output']['message'])
                if assistant_text and assistant_text not in result['assistant_messages']:
                    result['assistant_messages'].append(assistant_text)
    
    return result

def extract_text_content(content):
    """Helper function to extract text from various content formats"""
    if not content:
        return ""
    
    # Try to parse JSON content
    if isinstance(content, str) and (content.startswith('[') or content.startswith('{')):
        try:
            parsed = json.loads(content)
            
            # Handle list of items with text fields
            if isinstance(parsed, list):
                text = ""
                for item in parsed:
                    if isinstance(item, dict) and 'text' in item:
                        text += item['text']
                return text
            
            # Handle dictionary with text field
            elif isinstance(parsed, dict) and 'text' in parsed:
                return parsed['text']
            
        except (json.JSONDecodeError, TypeError):
            pass
    
    # Return content as is if parsing fails
    return content

In [8]:
def compliance_checking(conversation: str, compliance_rules: str, 
                       judge_id: str = 'anthropic.claude-3-haiku-20240307-v1:0',
                       max_tokens: int = 4096,
                       top_k: int = 50,
                       top_p: float = 0.1,
                       temperature: float = 0.1) -> str:
    """
   Compliance checking using an LLM judge.

    """
    # Initialize AWS Bedrock client
    bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")
   
    # Prepare prompt
    resp_fmt = """{
                   "score":float,
                   "reasoning": str
               }
           """

    user_prompt =  """
You are an expert compliance auditor tasked with evaluating agent-user conversations against specific compliance rules. Your role is to meticulously review the entire conversation and assess compliance with each provided rule.

## INSTRUCTIONS:

1. **Carefully read the entire conversation** between the agent and user from start to finish
2. **Analyze each compliance rule** provided in the compliance list
3. **Evaluate the agent's behavior** against each rule throughout the conversation
4. **Calculate an overall compliance score** based on adherence to all rules
5. **Provide brief reasoning** for your score

## EVALUATION CRITERIA:

- **1.0**: Perfect compliance - all rules followed completely
- **0.8-0.9**: High compliance - rules mostly followed with minor gaps
- **0.6-0.7**: Moderate compliance - some rules violated but core principles maintained
- **0.4-0.5**: Low compliance - significant violations of multiple rules
- **0.0-0.3**: Poor compliance - major violations or disregard for rules

## OUTPUT FORMAT: Provide output in a json format as follows:
{{COMPLIANCE SCORE: x%,
  REASONING: 2-3 short sentences explaining the score, highlighting key compliance successes or failures, and noting any critical violations that influenced the rating}}
  

## EVALUATION GUIDELINES:

- Focus only on AGENT behavior and responses, not user actions
- Consider the entire conversation flow and all provided compliance rules
- Weight critical compliance rules (safety, legal, ethical) more heavily than procedural ones
- Be objective and base scores on specific evidence from the conversation
- Factor in both violations and successful compliance demonstrations
- Do not add anything to the json output

Now, please evaluate the following conversation against the provided compliance rules:

**CONVERSATION:**
{conversation}
**COMPLIANCE RULES:**
{compliance_rules}
"""

    prompt = user_prompt.format(conversation=conversation,
                                compliance_rules=compliance_rules)

    # Prepare request body
    body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "messages": [{
            "role": "user",
            "content": [{
                "type": "text",
                "text": prompt
            }]
        }],
        "top_k": top_k,
        "top_p": top_p,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stop_sequences": ["Human"],
    })

        
    response = bedrock_client.invoke_model(
        modelId=judge_id,
        body=body,
        accept='application/json',
        contentType='application/json'
    )
       
    out1 = json.loads(response.get('body').read())
    response = out1.get('content')[0]['text'].replace("\n","")
    # Extract score
    score_match = re.search(r'COMPLIANCE SCORE:\s*([0-9]*\.?[0-9]+)', response)
    score = float(score_match.group(1)) if score_match else None
    
    # Extract reasoning
    reasoning_match = re.search(r'REASONING:\s*(.+)', response, re.DOTALL)
    reasoning = reasoning_match.group(1).strip() if reasoning_match else None
    
    return score, reasoning


## Results 

In [9]:
res_df = pd.DataFrame(columns = ['Questions', 'agent', 'Tools', 'Arguments', 
                                 'Expected Output',
                                 'called_tools','called_tools_args', 'called_tools_ans',
                                 'responses', 'final_answer', 'latency', 
                                 'input_tokens', 'output_tokens', 'ground_truths'])

### Q 10

In [10]:
idx = 10

trace_ids = {"s_a_s_t" : "1aa0100a7e917379e512aeb3d3ea46af", # single - single 
             "m_a_s_t" : "544e2242f3d467dee566f3f1de6b5820", # multi - single 
             "m_a_s_t_swarm" : "bc101c8abd5fb748b607e8362915accc"
            }

question = gt_data[idx]['question']
gt_tools = [i['name'] for i in gt_data[idx]['actions']]
args_dict = [i['arguments'] for i in gt_data[idx]['actions']]
gt_args = []
for i in args_dict:
    gt_args.append(list(i.values()))


In [11]:
for k in trace_ids:
    print(k)
    # Get trace data using your existing function
    trace_id = trace_ids[k]
    trace_data, observations = get_trace_with_observations(
        trace_id=trace_id,
        public_key=public_key,
        secret_key=secret_key,
        host=host
    )
    
    conversation_data = extract_tools_info_from_langfuse(trace_data, observations)

    latency = conversation_data['total_latency']
    input_tokens = conversation_data['input_tokens']
    output_tokens = conversation_data['output_tokens']
    called_tools = conversation_data['tools_list']
    called_tools_args = conversation_data['tools_args']
    called_tools_ans = conversation_data['tools_responses']
    responses = conversation_data['assistant_messages']
    final_answer = " ".join(responses)
    # print(called_tools)


    res_df.loc[len(res_df)] = [question, k, gt_tools, gt_args, None,
                 called_tools, called_tools_args, called_tools_ans,
                 responses, final_answer, latency,
                 input_tokens, output_tokens , [final_answer]]


s_a_s_t
Fetching trace data...
✅ Found 42 observations
m_a_s_t
Fetching trace data...
✅ Found 50 observations
m_a_s_t_swarm
Fetching trace data...
✅ Found 46 observations


### Q 14

In [12]:
idx = 14

trace_ids = {"s_a_s_t" : "8a0799fe6c8df76d29eacbb306698503", # single - single 
             "m_a_s_t" : "8372fcc695d1774f7f4f1a2870ffbf01", # multi - single 
             "m_a_s_t_swarm" : "233dc295526ddaf260e998b901bd0ed9"
            }

question = gt_data[idx]['question']
gt_tools = [i['name'] for i in gt_data[idx]['actions']]
args_dict = [i['arguments'] for i in gt_data[idx]['actions']]
gt_args = []
for i in args_dict:
    gt_args.append(list(i.values()))


In [13]:
for k in trace_ids:
    print(k)
    # Get trace data using your existing function
    trace_id = trace_ids[k]
    trace_data, observations = get_trace_with_observations(
        trace_id=trace_id,
        public_key=public_key,
        secret_key=secret_key,
        host=host
    )
    
    conversation_data = extract_tools_info_from_langfuse(trace_data, observations)

    latency = conversation_data['total_latency']
    input_tokens = conversation_data['input_tokens']
    output_tokens = conversation_data['output_tokens']
    called_tools = conversation_data['tools_list']
    called_tools_args = conversation_data['tools_args']
    called_tools_ans = conversation_data['tools_responses']
    responses = conversation_data['assistant_messages']
    final_answer = " ".join(responses)
    # print(called_tools)


    res_df.loc[len(res_df)] = [question, k, gt_tools, gt_args, None,
                 called_tools, called_tools_args, called_tools_ans,
                 responses, final_answer, latency,
                 input_tokens, output_tokens , [final_answer]]


s_a_s_t
Fetching trace data...
✅ Found 18 observations
m_a_s_t
Fetching trace data...
✅ Found 42 observations
m_a_s_t_swarm
Fetching trace data...
✅ Found 22 observations


### Q 19

In [14]:
idx = 19

trace_ids = {"s_a_s_t" : "8e6cf12d11608ecb1da9d02bfd8d1d34", # single - single 
             "m_a_s_t" : "11245034047d1aec1f755388c4d8c612", # multi - single 
             "m_a_s_t_swarm" : "a8ab878c0cc139d9a362fbc32d83a503"
            }

question = gt_data[idx]['question']
gt_tools = [i['name'] for i in gt_data[idx]['actions']]
args_dict = [i['arguments'] for i in gt_data[idx]['actions']]
gt_args = []
for i in args_dict:
    gt_args.append(list(i.values()))


In [15]:
for k in trace_ids:
    print(k)
    # Get trace data using your existing function
    trace_id = trace_ids[k]
    trace_data, observations = get_trace_with_observations(
        trace_id=trace_id,
        public_key=public_key,
        secret_key=secret_key,
        host=host
    )
    
    conversation_data = extract_tools_info_from_langfuse(trace_data, observations)

    latency = conversation_data['total_latency']
    input_tokens = conversation_data['input_tokens']
    output_tokens = conversation_data['output_tokens']
    called_tools = conversation_data['tools_list']
    called_tools_args = conversation_data['tools_args']
    called_tools_ans = conversation_data['tools_responses']
    responses = conversation_data['assistant_messages']
    final_answer = " ".join(responses)
    # print(called_tools)


    res_df.loc[len(res_df)] = [question, k, gt_tools, gt_args, None,
                 called_tools, called_tools_args, called_tools_ans,
                 responses, final_answer, latency,
                 input_tokens, output_tokens , [final_answer]]


s_a_s_t
Fetching trace data...
✅ Found 18 observations
m_a_s_t
Fetching trace data...
✅ Found 42 observations
m_a_s_t_swarm
Fetching trace data...
✅ Found 37 observations


### Q 20

In [16]:
idx = 20

trace_ids = {"s_a_s_t" : "e2c7e9bc415b6735c58fecd4d9bc7e78", # single - single - Q 20
             "m_a_s_t" : "83ab5f9ebc6a9b41ca1a4dfc917225dc", # multi - single - Q 20
             "m_a_s_t_swarm" : "932fcef76a0faa66189a077d09a19e60"
            }

question = gt_data[idx]['question']
gt_tools = [i['name'] for i in gt_data[idx]['actions']]
args_dict = [i['arguments'] for i in gt_data[idx]['actions']]
gt_args = []
for i in args_dict:
    gt_args.append(list(i.values()))



In [17]:
for k in trace_ids:
    print(k)
    # Get trace data using your existing function
    trace_id = trace_ids[k]
    trace_data, observations = get_trace_with_observations(
        trace_id=trace_id,
        public_key=public_key,
        secret_key=secret_key,
        host=host
    )
    
    conversation_data = extract_tools_info_from_langfuse(trace_data, observations)

    latency = conversation_data['total_latency']
    input_tokens = conversation_data['input_tokens']
    output_tokens = conversation_data['output_tokens']
    called_tools = conversation_data['tools_list']
    called_tools_args = conversation_data['tools_args']
    called_tools_ans = conversation_data['tools_responses']
    responses = conversation_data['assistant_messages']
    final_answer = " ".join(responses)
    # print(called_tools)


    res_df.loc[len(res_df)] = [question, k, gt_tools, gt_args, None,
                 called_tools, called_tools_args, called_tools_ans,
                 responses, final_answer, latency,
                 input_tokens, output_tokens , [final_answer]]


s_a_s_t
Fetching trace data...
✅ Found 15 observations
m_a_s_t
Fetching trace data...
✅ Found 50 observations
m_a_s_t_swarm
Fetching trace data...
✅ Found 50 observations


## Evaluation

In [18]:
metric_list = ["missed_tool_pct",
               "incorrect_tool_pct",
               "tools_args_acc",
               "answer_relevancy",
               "compliance_score",
               "compliance_reasoning",
               "latency",
               "input_tokens",
               "output_tokens",
              ]
available_tools = " "
eval_res = calc_metrics(res_df, metric_list, available_tools=available_tools)

eval_res["compliance_score"] = eval_res["responses"].apply(
    lambda x: compliance_checking(" ".join(x), compliance_rules)[0]
)
eval_res["compliance_reasoning"] = eval_res["responses"].apply(
    lambda x: compliance_checking(" ".join(x), compliance_rules)[1]
)

##### available_tools  


In [19]:
eval_res[["Questions", "agent"]+metric_list]

,Questions,agent,missed_tool_pct,incorrect_tool_pct,tools_args_acc,answer_relevancy,compliance_score,compliance_reasoning,latency,input_tokens,output_tokens
0,"\nMy user id is mia_kim_4397. I want to remove Ethan from my reservation H9ZU1C. If the change is not possible, I want you to cancel it, and I will rebook myself. I am also looking for the cheapest direct round trip flight from New York (either EWR or JFK) to anywhere on the West Coast, with a departure date of May 20 and a return date of May 25. I am fine with basic economy class if it is cheaper. Please book it for me. I want to first use up my smaller gift card and then the larger one. I want to use all my free baggage allowance but no insurance. My date of birth is in my user profile, and I do not want to speak it. I also wonder why cancellation does not refund to a gift card now.\n",s_a_s_t,0.0,0.846154,0.666667,0.8,0.8,"The agent demonstrated high compliance with the provided rules, with a few minor violations. The agent successfully navigated the flight change and cancellation policies, explaining the limitations and options available to the user. However, the agent made a mistake in incorrectly stating that the user had a credit card on file, which is a procedural violation. Overall, the agent maintained a strong focus on finding a solution within the user's constraints and the compliance rules.}",250.575,210842,3952
1,"\nMy user id is mia_kim_4397. I want to remove Ethan from my reservation H9ZU1C. If the change is not possible, I want you to cancel it, and I will rebook myself. I am also looking for the cheapest direct round trip flight from New York (either EWR or JFK) to anywhere on the West Coast, with a departure date of May 20 and a return date of May 25. I am fine with basic economy class if it is cheaper. Please book it for me. I want to first use up my smaller gift card and then the larger one. I want to use all my free baggage allowance but no insurance. My date of birth is in my user profile, and I do not want to speak it. I also wonder why cancellation does not refund to a gift card now.\n",m_a_s_t,0.5,0.928571,0.454545,0.5,0.6,"The agent demonstrated moderate compliance with the provided compliance rules. While they were able to search for and identify one-stop flight options from the New York area to destinations in California, Oregon, Washington, and Nevada, they struggled to actually book the requested flights due to limitations in flight availability. This suggests potential gaps in the agent's understanding of the specific booking constraints for basic economy fares on these routes. Additionally, the agent did not provide clear explanations of the airline's cancellation and refund policies, which are critical compliance requirements. Overall, the agent maintained the core principles of the compliance rules but had significant violations that impacted the final compliance score.}",384.552,180505,5589
2,"\nMy user id is mia_kim_4397. I want to remove Ethan from my reservation H9ZU1C. If the change is not possible, I want you to cancel it, and I will rebook myself. I am also looking for the cheapest direct round trip flight from New York (either EWR or JFK) to anywhere on the West Coast, with a departure date of May 20 and a return date of May 25. I am fine with basic economy class if it is cheaper. Please book it for me. I want to first use up my smaller gift card and then the larger one. I want to use all my free baggage allowance but no insurance. My date of birth is in my user profile, and I do not want to speak it. I also wonder why cancellation does not refund to a gift card now.\n",m_a_s_t_swarm,0.5,0.916667,1.000000,1.0,0.9,"The agent demonstrated high compliance with the provided compliance rules throughout the conversation. They clearly explained the airline's policies around refunds, changes, and baggage, showing a strong understanding of the relevant rules. The agent handled the reservation cancellation and new flight booking appropri